# TENSORFLOW SAVERS AND SUMMARY WRITERS

### STEP 1: IMPORT LIBRARIES

In [1]:
import tensorflow as tf

LOGDIR = './logs/10-savers/logs/'

In [2]:
sess = tf.InteractiveSession()

### STEP 2: PREPARE DATASET

- ```y = x * W + b```

In [3]:
train_x = [ i for i in range(10) ]
train_y = [ i * 3 + 10 for i in range(10) ] 
test_x = [ i for i in range(50, 70, 7) ]
test_y = [ i * 3 + 10 for i in range(50, 70, 7) ]

### STEP 3: DEFINE THE SHAPE OF OUR DATASET

In [4]:
x = tf.placeholder(tf.float32, name='X')
y_ = tf.placeholder(tf.float32,  name='Y_')

### STEP 4: DEFINE HELPER FUNCTIONS

### STEP 5: DEFINE MODEL

In [5]:
W = tf.Variable([.3], tf.float32, name='W')
b = tf.Variable([-.3], tf.float32, name='b')

y = tf.add(tf.multiply(W, x), b, name='Y')

### STEP 6: DEFINE METRICS

#### 1. LOSS

In [6]:
loss = tf.reduce_sum(tf.square(y - y_))
tf.summary.scalar('LOSS', loss)

<tf.Tensor 'LOSS:0' shape=() dtype=string>

#### 2. TRAINING USING GRADIENT DESCENT OPTIMIZER

In [7]:
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

#### 3. ACCURACY

In [8]:
accuracy = 1 - tf.reduce_mean(tf.cast(((y - y_) / y_) ** 2, tf.float32))
tf.summary.scalar('ACCURACY', accuracy)

<tf.Tensor 'ACCURACY:0' shape=() dtype=string>

### STEP 7: TRAIN MODEL

#### CREATE SAVERS AND SUMMARY WRITERS

In [9]:
summary = tf.summary.merge_all()
writer_train = tf.summary.FileWriter(LOGDIR+'train/')
writer_test = tf.summary.FileWriter(LOGDIR+'test/')

writer_train.add_graph(sess.graph)
writer_test.add_graph(sess.graph)

#### LOAD CHECKPOINTS IF AVAILABLE

In [10]:
checkpoint = tf.train.latest_checkpoint(checkpoint_dir=LOGDIR)

saver = tf.train.Saver(tf.global_variables())

if checkpoint:
    try:
        saver.restore(sess, tf.train.latest_checkpoint(checkpoint_dir=LOGDIR))
        
        for i in tf.trainable_variables():
            sess.run(i); print('LOADED:', i)
    except: print('err')
else:
    try:
        sess.run(tf.global_variables_initializer())
    except:
        sess.run(tf.initialize_all_variables())

INFO:tensorflow:Restoring parameters from ./logs/10-savers/logs/-900-.ckpt
LOADED: <tf.Variable 'W:0' shape=(1,) dtype=float32_ref>
LOADED: <tf.Variable 'b:0' shape=(1,) dtype=float32_ref>


In [11]:
for i in range(1000):
    _, summ = sess.run([train_step, summary], feed_dict={x: train_x, y_: train_y})

    writer_train.add_summary(summ, i)
    
    if i % 100 == 0:
        saver.save(sess, LOGDIR +  '-' + str(i) + '-' + '.ckpt')

        curr_loss, curr_accuracy, summ = sess.run([loss, accuracy, summary], feed_dict={x: train_x, y_: train_y})

        writer_test.add_summary(summ, i)
        
        print('STEP %3d`s ACCURACY %f, LOSS %f' % (i, curr_accuracy, curr_loss))

STEP   0`s ACCURACY 1.000000, LOSS 0.000000
STEP 100`s ACCURACY 1.000000, LOSS 0.000000
STEP 200`s ACCURACY 1.000000, LOSS 0.000000
STEP 300`s ACCURACY 1.000000, LOSS 0.000000
STEP 400`s ACCURACY 1.000000, LOSS 0.000000
STEP 500`s ACCURACY 1.000000, LOSS 0.000000
STEP 600`s ACCURACY 1.000000, LOSS 0.000000
STEP 700`s ACCURACY 1.000000, LOSS 0.000000
STEP 800`s ACCURACY 1.000000, LOSS 0.000000
STEP 900`s ACCURACY 1.000000, LOSS 0.000000


### STEP 8: EVALUATE MODEL

In [12]:
final_loss, final_accuracy = sess.run([loss, accuracy], feed_dict={x: test_x, y_: test_y})

print('FINAL ACCURACY IS %f and LOSS IS %f' % (final_accuracy, final_loss))

FINAL ACCURACY IS 1.000000 and LOSS IS 0.000001


### STEP 9: TEST PREDICTION

In [13]:
x_actual = 12
y_actual = x_actual * 3 + 10
y_predict = sess.run(y, feed_dict={x: x_actual})

print('ACTUAL VALUE:', y_actual)
print('PREDICTED VALUE:', y_predict)

ACTUAL VALUE: 46
PREDICTED VALUE: [ 46.00008011]
